In [1]:
import os
import langchain
import gradio as gr

In [2]:
# Load content from Resume Document 
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/data/SowmyaYellapragada-Resume.pdf")
documents = loader.load()

In [3]:
# Split the Document into chunks for embedding and vector storage.
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 600, chunk_overlap = 0)
all_splits = text_splitter.split_documents(documents)

In [4]:
import os
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.vectorstores import Qdrant
model_1_embeddings = CohereEmbeddings(model = "multilingual-22-12")

# Compute embeddings of the text splits
db = Qdrant.from_documents(all_splits, model_1_embeddings, location=":memory:", collection_name="all_splits", distance_func="Dot")

In [5]:
import langchain
langchain.debug = False

from langchain.chains import RetrievalQA
from langchain.llms import Cohere
model_2_qa = RetrievalQA.from_chain_type(llm=Cohere(model="command-nightly", temperature=0.9),
                                         chain_type="stuff",
                                         retriever=db.as_retriever(),
                                         return_source_documents=False)

In [6]:
questions = [
           "Whats the name of the resume holder",
           "What is she experienced in?", 
           "Where did she work?"
           ]

In [7]:
for question in questions:
    answer = model_2_qa({"query": question})
    print(answer["query"])
    print(answer["result"])
    print("\n")
    

/usr/local/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Whats the name of the resume holder
The name of the resume holder is Sowmya Yellapragada. This can be found under the "About Me" section of the resume. 


What is she experienced in?
Based on the provided information, Sowmya has experience as a software engineer with a focus on machine learning. Her experience includes designing and implementing machine learning infrastructure platforms, setting up data pipelines, developing microservices, and leading teams. She also has experience with specific software systems, languages, and tools such as ArgoCD, Concourse, Golang, and Git, among others.  Her record of designing and implementing an ML infrastructure platform and her patent in predicting travel time show her expertise and contribution to the field.  It might be helpful to clarify any specific area of experience you are interested in to give a more detailed answer.  I don't know is an acceptable answer and might be the best approach given the general nature of your question.   To gain

In [8]:
from langchain.llms import Cohere
llm = Cohere(model = "command-nightly", temperature=0.9)

In [9]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.conversation.memory import ConversationSummaryMemory
#Setup Chat History
#The chat_history variable keeps track of the conversation history, storing the user queries #and the chatbot's responses
#This allows the chatbot to maintain context and provide relevant answers based on past interactions.
chat_history = []
model_3_qa = ConversationalRetrievalChain.from_llm(
    llm = llm,
    chain_type = "stuff",
    memory = ConversationSummaryMemory(llm = llm, memory_key='chat_history', input_key='question', output_key= 'answer', return_messages=True),
    retriever = db.as_retriever(),
    return_source_documents=False
)

In [10]:
def QandA(question):
    global chat_history
    result = model_3_qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result['answer']))
    return result['answer']

In [11]:
#DocumentQA Interface
DocQABotUI = gr.Interface(
    fn=QandA,
    inputs="text",
    outputs="text",
    title="Chatbot",
    description="Document QA"
)
#Launching the DocQABot
DocQABotUI.launch(share=True,server_port=8889)

Running on local URL:  http://127.0.0.1:8889
Running on public URL: https://81c19218a6ba798172.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Retrying langchain_community.llms.cohere.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised TooManyRequestsError: status_code: 429, body: {'message': "You are using a Trial key, which is limited to 5 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"}.
Retrying langchain_community.llms.cohere.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised TooManyRequestsError: status_code: 429, body: {'message': "You are using a Trial key, which is limited to 5 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"}.
Re